In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import statsmodels.api as spi
import datetime as dt 
import yfinance as yf
import pandas_ta 
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-11-29'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,start=start_date,end=end_date)
newdf = df
newdf = newdf.stack()
newdf.index.names = ['date','ticker']
newdf.columns = newdf.columns.str.lower()
newdf

[*********************100%***********************]  502 of 502 completed


Price                              adj close       close        high  \
date                      ticker                                       
2016-12-01 00:00:00+00:00 A        40.615158   43.209999   44.099998   
                          AAPL     25.349981   27.372499   27.735001   
                          ABBV     41.960175   59.160000   60.830002   
                          ABT      32.533455   37.599998   38.299999   
                          ACGL     26.403282   27.766666   27.906668   
...                                      ...         ...         ...   
2024-11-27 00:00:00+00:00 XYL     125.610001  125.610001  127.080002   
                          YUM     138.658112  139.330002  139.979996   
                          ZBH     112.019997  112.019997  112.660004   
                          ZBRA    405.750000  405.750000  406.140015   
                          ZTS     176.740005  176.740005  179.270004   

Price                                    low        open       volume  
date                      ticker                                       
2016-12-01 00:00:00+00:00 A        42.919998   44.080002    2823100.0  
                          AAPL     27.257500   27.592501  148347600.0  
                          ABBV     58.750000   60.830002    8266200.0  
                          ABT      37.380001   38.119999    9008500.0  
                          ACGL     27.633333   27.696667    1493700.0  
...                                      ...         ...          ...  
2024-11-27 00:00:00+00:00 XYL     124.930000  125.440002    1225200.0  
                          YUM     138.690002  138.820007    1614100.0  
                          ZBH     110.510002  110.839996    1246000.0  
                          ZBRA    400.929993  403.730011     258300.0  
                          ZTS     175.000000  175.270004    2315800.0  

[987320 rows x 6 columns]

# Calculating Features and technical Indicators For each stock
### 1: Garman-Klass Volatility
### Garman-Klass Volatility = ((ln(High) - ln(Low))^2 / 2) - (2 ln(2) - 1) (ln(Adj Close) - ln(Open))^2


In [2]:
newdf['garman_klass_vol'] = ((np.log(newdf['high']) - np.log(newdf['low']))**2 / 2) - ((2 * np.log(2) - 1) * (np.log(newdf['adj close']) - np.log(newdf['open']))**2)
newdf

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-08-01 A        44.955276   47.869999   48.110001   47.619999   48.110001   
           AAL      34.212429   35.599998   35.750000   35.099998   35.540001   
           AAPL     24.351376   26.512501   26.537500   26.102501   26.102501   
           ABBV     47.420174   66.940002   67.080002   66.230003   66.230003   
           ABT      39.123219   45.290001   45.580002   44.720001   44.750000   
...                       ...         ...         ...         ...         ...   
2024-07-26 XYL     140.839996  140.839996  142.130005  137.820007  138.479996   
           YUM     128.050003  128.050003  129.039993  127.410004  127.690002   
           ZBH     111.290001  111.290001  112.279999  110.230003  110.790001   
           ZBRA    325.980011  325.980011  330.970001  323.000000  326.500000   
           ZTS     179.839996  179.839996  183.850006  179.259995  182.259995   

Price                   volume  garman_klass_vol  
date       ticker                                 
2016-08-01 A         1604600.0         -0.001724  
           AAL       7059000.0         -0.000392  
           AAPL    152671600.0         -0.001726  
           ABBV      7599000.0         -0.043034  
           ABT      12152400.0         -0.006794  
...                        ...               ...  
2024-07-26 XYL       1074100.0          0.000364  
           YUM       1874400.0          0.000078  
           ZBH       1399400.0          0.000162  
           ZBRA       458700.0          0.000296  
           ZTS       2437300.0          0.000251  

[989766 rows x 7 columns]

In [3]:
# RSI Indicator 
newdf['rsi'] = newdf.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x,length=20))
# Bollinger bands Indicator
def calculate_bbands(series, length=20):
    if len(series) < length:
        return pd.DataFrame(index=series.index, columns=['BB_Lower', 'BB_Mid', 'BB_Upper'])
    else:
        bbands = pandas_ta.bbands(close=np.log1p(series), length=length)
        if bbands is None or bbands.empty:
            return pd.DataFrame(index=series.index, columns=['BB_Lower', 'BB_Mid', 'BB_Upper'])
        return bbands
newdf['bb low'] = newdf.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:, 0])
newdf['bb mid'] = newdf.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:, 1])
newdf['bb high'] = newdf.groupby(level=1)['adj close'].transform(lambda x: calculate_bbands(x).iloc[:, 2])
newdf.sample(5)


,Price,adj close,close,high,low,open,volume,garman_klass_vol,rsi,bb low,bb mid,bb high
date,ticker,,,,,,,,,,,
2020-12-22,FE,25.903925,29.950001,30.090000,29.700001,29.900000,4139600.0,-0.007866,52.415890,3.15252,3.263456,3.374391
2023-05-09,NI,27.372677,28.450001,28.540001,28.230000,28.340000,3047900.0,-0.000406,55.326142,3.32205,3.338906,3.355762
2017-09-28,AAPL,36.011280,38.320000,38.570000,38.174999,38.472500,88022000.0,-0.001635,44.103739,3.591486,3.641069,3.690652
2018-03-15,CE,92.212921,105.730003,106.849998,105.660004,106.169998,448000.0,-0.007611,50.806404,4.482931,4.525705,4.56848
2023-11-02,GILD,78.715302,81.230003,81.309998,79.639999,79.809998,6166100.0,0.000142,63.152033,4.296025,4.337983,4.37994


In [4]:
# Average True Range (ATR)
def calculate_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],low=stock_data['low'],close=stock_data['close'],length=14)
    return  atr.sub(atr.mean()).div(atr.std())
newdf['atr'] = newdf.groupby(level=1,group_keys=False).apply(calculate_atr)
newdf.sample(5)

,Price,adj close,close,high,low,open,volume,garman_klass_vol,rsi,bb low,bb mid,bb high,atr
date,ticker,,,,,,,,,,,,
2023-08-25,ADSK,215.919998,215.919998,216.800003,209.490005,209.850006,1933700.0,0.000274,58.385970,5.287496,5.331909,5.376321,-0.000301
2021-12-21,CI,211.017136,220.240005,221.649994,216.000000,216.000000,1570600.0,0.000123,59.495421,5.216154,5.302551,5.388948,-0.002071
2022-12-22,ROL,36.207661,36.970001,37.049999,36.520000,37.049999,1198700.0,-0.000101,38.772063,3.596645,3.660648,3.724651,0.729118
2017-12-22,GEN,14.362357,28.320000,28.610001,28.059999,28.480000,2643900.0,-0.180854,44.959706,2.699068,2.735983,2.772899,0.322848
2019-05-03,DD,67.025581,73.841270,74.603172,72.740738,73.714287,7001978.0,-0.003176,37.304558,4.238686,4.301688,4.36469,0.593063


In [5]:
# MACD Indicator
short_ema = newdf['adj close'].ewm(span=12, adjust=False).mean()
long_ema = newdf['adj close'].ewm(span=25,adjust = False).mean()
macd = short_ema - long_ema
macd = macd.sub(macd.mean())/macd.std()
newdf['macd'] = macd
newdf.sample(5)

,Price,adj close,close,high,low,open,volume,garman_klass_vol,rsi,bb low,bb mid,bb high,atr,macd
date,ticker,,,,,,,,,,,,,
2018-06-25,SMCI,24.100000,24.100000,24.900000,23.860001,24.799999,328200.0,0.000593,51.959695,3.202403,3.269761,3.337119,-0.363013,-0.299563
2021-08-02,BBWI,61.145596,64.607925,66.289413,64.058205,65.198059,5282856.0,-0.001005,64.146047,4.000405,4.072681,4.144957,1.219836,0.081179
2020-01-23,KDP,25.070913,27.770000,27.820000,27.139999,27.540001,5988100.0,-0.003102,43.979659,3.237952,3.273891,3.30983,-0.532694,-0.402920
2018-05-04,HPQ,17.773167,21.680000,21.780001,21.209999,21.209999,6547700.0,-0.011721,48.453761,2.903886,2.928633,2.953381,-0.587021,-0.224018
2019-05-03,ABT,71.900940,78.690002,79.129997,78.260002,78.830002,5452700.0,-0.003209,54.893718,4.227399,4.273068,4.318737,-0.451184,-0.119183


In [6]:
newdf['dollar_volume'] = (newdf['adj close'] * newdf['volume'])/1e6
newdf.head(3)

Price              adj close      close       high        low       open  \
date       ticker                                                          
2016-08-01 A       44.955276  47.869999  48.110001  47.619999  48.110001   
           AAL     34.212429  35.599998  35.750000  35.099998  35.540001   
           AAPL    24.351376  26.512501  26.537500  26.102501  26.102501   

Price                   volume  garman_klass_vol  rsi bb low bb mid bb high  \
date       ticker                                                             
2016-08-01 A         1604600.0         -0.001724  NaN    NaN    NaN     NaN   
           AAL       7059000.0         -0.000392  NaN    NaN    NaN     NaN   
           AAPL    152671600.0         -0.001726  NaN    NaN    NaN     NaN   

Price              atr      macd  dollar_volume  
date       ticker                                
2016-08-01 A       NaN -0.000021      72.135237  
           AAL     NaN -0.028038     241.505537  
           AAPL    NaN -0.075306    3717.763472

# Aggregating to Monthly Level and Filtering Top 150 most liquid stocks

In [7]:
last_cols = [c for c in newdf.columns.unique() if c not in ['dollar_volume','volume','high','low','open','close']]

data = pd.concat([newdf.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
newdf.unstack()[last_cols].resample('M').last().stack('ticker')],axis=1).dropna()
data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2016-08-31 A           87.662068   44.119476         -0.001640  44.201780   
           AAL        269.710266   34.989559         -0.000479  54.911528   
           AAPL      2718.975467   24.494833         -0.002162  43.809182   
           ABBV       306.400100   45.408325         -0.046259  25.042296   
           ABT        375.329241   36.298466         -0.010390  18.668343   
...                          ...         ...               ...        ...   
2024-07-31 XYL        138.716575  140.839996          0.000364  55.586848   
           YUM        243.471007  128.050003          0.000078  42.837561   
           ZBH        157.700232  111.290001          0.000162  52.282335   
           ZBRA        94.195502  325.980011          0.000296  56.303553   
           ZTS        336.254363  179.839996          0.000251  56.937363   

                     bb low    bb mid   bb high       atr      macd  
date       ticker                                                    
2016-08-31 A       3.799092  3.820731  3.842369 -1.391926 -0.022261  
           AAL     3.511264  3.567882    3.6245 -0.052030 -0.084046  
           AAPL    3.235305  3.255475  3.275645 -1.356712 -0.158680  
           ABBV    3.835019  3.867038  3.899057 -1.552746 -0.160555  
           ABT      3.62483  3.667518  3.710207 -1.212625 -0.183877  
...                     ...       ...       ...       ...       ...  
2024-07-31 XYL     4.883905  4.925051  4.966198  1.303668 -0.446098  
           YUM     4.840427  4.864791  4.889156  0.531118 -0.385216  
           ZBH     4.660673  4.696232  4.731791 -0.580008 -0.376814  
           ZBRA    5.724346  5.774209  5.824072 -0.068389  0.194101  
           ZTS     5.148637  5.183583  5.218529  0.714489  0.256605  

[47262 rows x 9 columns]

In [8]:
data['dollar_volume'] = data.loc[:,'dollar_volume'].unstack('ticker').rolling(5*12,min_periods=12).mean().stack()
data['dollar_rank'] = data.groupby('date')['dollar_volume'].rank(ascending=False)
data = data[data['dollar_rank']<150].drop(['dollar_volume','dollar_rank'],axis=1)
data

adj close  garman_klass_vol        rsi    bb low  \
date       ticker                                                      
2017-07-31 AAL      48.955460         -0.000281  49.738425  3.892343   
           AAPL     34.805637         -0.002001  50.672304  3.533293   
           ABBV     51.445614         -0.038363  45.182680  3.959512   
           ABT      43.532555         -0.006503  55.958971   3.75388   
           ACN     115.462814         -0.004393  60.835462  4.707508   
...                       ...               ...        ...       ...   
2024-07-31 VZ       40.090000          0.000076  48.896996  3.689697   
           WFC      60.389999          0.000076  54.116783  4.070207   
           WMT      69.779999          0.000190  58.845792  4.229259   
           XOM     117.330002          0.000080  56.064246  4.717199   
           ZTS     179.839996          0.000251  56.937363  5.148637   

                     bb mid   bb high       atr      macd  
date       ticker                                          
2017-07-31 AAL       3.9412  3.990058  0.811088 -0.036637  
           AAPL     3.57507  3.616847 -1.171233 -0.117636  
           ABBV    3.990966  4.022419 -1.309358 -0.136114  
           ABT     3.796236  3.838591 -1.350002 -0.169573  
           ACN     4.740108  4.772708 -1.271110 -0.044957  
...                     ...       ...       ...       ...  
2024-07-31 VZ      3.730417  3.771136 -0.001174 -1.125652  
           WFC      4.10434  4.138474  0.752182 -1.262914  
           WMT     4.258481  4.287703  0.823370 -1.081383  
           XOM     4.751784  4.786369  0.644545 -0.559990  
           ZTS     5.183583  5.218529  0.714489  0.256605  

[12665 rows x 8 columns]

# Calculate Monthly Returns for different time horizons as features

In [9]:
def calculate_returns(df):
    outlier_cutoff = .05
    lags = [1,2,3,6,9,12]

    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close'].pct_change(lag).pipe(lambda x: x.clip
        (lower = x.quantile(outlier_cutoff)
        ,upper = x.quantile(1-outlier_cutoff)))
        .add(1).pow(1/lag).sub(1))
    return df

data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
data.tail(5)


adj close  garman_klass_vol        rsi    bb low  \
date       ticker                                                      
2024-07-31 VRTX    495.260010          0.000228  65.124254  6.151693   
           VZ       40.090000          0.000076  48.896996  3.689697   
           WFC      60.389999          0.000076  54.116783  4.070207   
           WMT      69.779999          0.000190  58.845792  4.229259   
           XOM     117.330002          0.000080  56.064246  4.717199   

                     bb mid   bb high       atr      macd  return_1m  \
date       ticker                                                      
2024-07-31 VRTX    6.187225  6.222756  1.329312  0.647954   0.056622   
           VZ      3.730417  3.771136 -0.001174 -1.125652  -0.011977   
           WFC      4.10434  4.138474  0.752182 -1.262914   0.016838   
           WMT     4.258481  4.287703  0.823370 -1.081383   0.030572   
           XOM     4.751784  4.786369  0.644545 -0.559990   0.019197   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2024-07-31 VRTX     0.042915   0.071946   0.022494   0.035405    0.028780  
           VZ      -0.004920   0.010492  -0.003750   0.020325    0.018346  
           WFC      0.003914   0.007901   0.033540   0.050024    0.025153  
           WMT      0.030112   0.054515   0.041393   0.028143    0.022880  
           XOM      0.000298   0.000037   0.025221   0.014483    0.010431

# Fama French Factor

In [10]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',
               start='2010')[0].drop('RF',axis = 1)
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)
factor_data.index.name = 'date'
factor_data = factor_data.join(data['return_1m']).sort_index()
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-07-31 AAL     0.0319 -0.0193  0.0047  0.0156  0.0035   0.041623
           AAPL    0.0319 -0.0193  0.0047  0.0156  0.0035   0.027984
           ABBV    0.0319 -0.0193  0.0047  0.0156  0.0035   0.005634
           ABT     0.0319 -0.0193  0.0047  0.0156  0.0035   0.079433
           ACN     0.0319 -0.0193  0.0047  0.0156  0.0035  -0.026041
...                   ...     ...     ...     ...     ...        ...
2024-05-31 VRTX    0.0434  0.0059 -0.0133  0.0321 -0.0287   0.146716
           VZ      0.0434  0.0059 -0.0133  0.0321 -0.0287   0.042036
           WFC     0.0434  0.0059 -0.0133  0.0321 -0.0287   0.015922
           WMT     0.0434  0.0059 -0.0133  0.0321 -0.0287   0.100618
           XOM     0.0434  0.0059 -0.0133  0.0321 -0.0287  -0.000487

[10222 rows x 6 columns]

In [11]:
observations = factor_data.groupby(level=1).size()
# observations
valid_stocks = observations[observations>=10]
factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-07-31 AAL     0.0319 -0.0193  0.0047  0.0156  0.0035   0.041623
           AAPL    0.0319 -0.0193  0.0047  0.0156  0.0035   0.027984
           ABBV    0.0319 -0.0193  0.0047  0.0156  0.0035   0.005634
           ABT     0.0319 -0.0193  0.0047  0.0156  0.0035   0.079433
           ACN     0.0319 -0.0193  0.0047  0.0156  0.0035  -0.026041
...                   ...     ...     ...     ...     ...        ...
2024-05-31 VRTX    0.0434  0.0059 -0.0133  0.0321 -0.0287   0.146716
           VZ      0.0434  0.0059 -0.0133  0.0321 -0.0287   0.042036
           WFC     0.0434  0.0059 -0.0133  0.0321 -0.0287   0.015922
           WMT     0.0434  0.0059 -0.0133  0.0321 -0.0287   0.100618
           XOM     0.0434  0.0059 -0.0133  0.0321 -0.0287  -0.000487

[10181 rows x 6 columns]

In [12]:
betas = (factor_data.groupby(level=1,group_keys=False).apply(
        lambda x : RollingOLS(endog=x['return_1m'],exog = spi.add_constant(x.drop('return_1m',axis =1)),
                        window=min(24,x.shape[0]),
                        min_nobs=len(x.columns)+1)
                        .fit(params_only = True)
                        .params
                        .drop('const',axis =1)))

betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2018-07-31 AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2024-05-31 VRTX    0.381685  0.762846 -0.580974  0.625204  0.523176
           VZ      0.438396 -0.599228  0.107771  0.284499  0.189511
           WFC     1.125169  0.109168  0.972288 -0.637281 -0.528380
           WMT     0.565130 -0.033600 -0.020386  0.739701  0.044140
           XOM     0.737146 -0.178147  0.442600  0.620364  0.337120

[10181 rows x 5 columns]

In [13]:
factors = ["Mkt-RF","SMB","HML","RMW","CMA"]

data = data.join(betas.groupby('ticker').shift())
data.loc[:,factors] = data.groupby('ticker',group_keys = False)[factors].apply(lambda x:x.fillna(x.mean()))
data.dropna()
data = data.drop('adj close',axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10506 entries, (Timestamp('2018-07-31 00:00:00'), 'AAL') to (Timestamp('2024-07-31 00:00:00'), 'XOM')
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   garman_klass_vol  10506 non-null  float64
 1   rsi               10506 non-null  float64
 2   bb low            10506 non-null  object 
 3   bb mid            10506 non-null  object 
 4   bb high           10506 non-null  object 
 5   atr               10506 non-null  float64
 6   macd              10506 non-null  float64
 7   return_1m         10506 non-null  float64
 8   return_2m         10506 non-null  float64
 9   return_3m         10506 non-null  float64
 10  return_6m         10506 non-null  float64
 11  return_9m         10506 non-null  float64
 12  return_12m        10506 non-null  float64
 13  Mkt-RF            10306 non-null  float64
 14  SMB               10306 non-null  float64
 15  HML        

In [14]:
data

garman_klass_vol        rsi    bb low    bb mid   bb high  \
date       ticker                                                              
2018-07-31 AAL             0.000092  49.117697   3.59191  3.645691  3.699472   
           AAPL           -0.000885  52.605362  3.808181  3.834322  3.860463   
           ABBV           -0.027194  45.920966   4.22565  4.279128  4.332605   
           ABT            -0.003867  62.882561  4.015925  4.062819  4.109713   
           ACN            -0.003530  42.887792  4.993172  5.023605  5.054037   
...                             ...        ...       ...       ...       ...   
2024-07-31 VRTX            0.000228  65.124254  6.151693  6.187225  6.222756   
           VZ              0.000076  48.896996  3.689697  3.730417  3.771136   
           WFC             0.000076  54.116783  4.070207   4.10434  4.138474   
           WMT             0.000190  58.845792  4.229259  4.258481  4.287703   
           XOM             0.000080  56.064246  4.717199  4.751784  4.786369   

                        atr      macd  return_1m  return_2m  return_3m  \
date       ticker                                                        
2018-07-31 AAL     0.806344 -0.049969   0.041623  -0.047041  -0.026289   
           AAPL   -1.094848 -0.125222   0.027984   0.009109   0.049478   
           ABBV    0.022405 -0.117232   0.005634  -0.029587  -0.011801   
           ABT    -0.961629 -0.138642   0.079433   0.034389   0.042361   
           ACN    -0.857337  0.004187  -0.026041   0.011460   0.017611   
...                     ...       ...        ...        ...        ...   
2024-07-31 VRTX    1.329312  0.647954   0.056622   0.042915   0.071946   
           VZ     -0.001174 -1.125652  -0.011977  -0.004920   0.010492   
           WFC     0.752182 -1.262914   0.016838   0.003914   0.007901   
           WMT     0.823370 -1.081383   0.030572   0.030112   0.054515   
           XOM     0.644545 -0.559990   0.019197   0.000298   0.000037   

                   return_6m  return_9m  return_12m    Mkt-RF       SMB  \
date       ticker                                                         
2018-07-31 AAL     -0.050880  -0.017899   -0.019393  0.934340  1.608378   
           AAPL     0.022909   0.014540    0.022060  1.286020 -0.358114   
           ABBV    -0.028188   0.005495    0.026310  0.415843  0.466637   
           ABT      0.010427   0.022867    0.025837  0.729700 -0.216381   
           ACN      0.000037   0.013581    0.019435  1.147167 -0.288425   
...                      ...        ...         ...       ...       ...   
2024-07-31 VRTX     0.022494   0.035405    0.028780  0.426693  0.183769   
           VZ      -0.003750   0.020325    0.018346  0.451346 -0.414833   
           WFC      0.033540   0.050024    0.025153  0.938055 -0.256432   
           WMT      0.041393   0.028143    0.022880  0.615487 -0.416216   
           XOM      0.025221   0.014483    0.010431  0.944041  0.166029   

                        HML       RMW       CMA  
date       ticker                                
2018-07-31 AAL     0.190539  0.398459  0.807292  
           AAPL   -0.606317  0.252990  0.681816  
           ABBV   -0.190511  0.614825  0.415059  
           ABT    -0.348493  0.148103  0.770405  
           ACN    -0.166731  0.285844 -0.029701  
...                     ...       ...       ...  
2024-07-31 VRTX   -0.817312  0.084487  0.702343  
           VZ      0.182801  0.181548  0.248143  
           WFC     1.249730 -0.571861 -0.546576  
           WMT    -0.238909  0.206231  0.584845  
           XOM     0.615648 -0.476545  0.556692  

[10506 rows x 18 columns]

In [19]:
from sklearn.cluster import KMeans

def get_clusters(df):
    df['clusters'] = KMeans(n_clusters=4,
                            init='random',
                            random_state=0).fit(df).labels_
    
    return df

data = data.dropna().groupby('date',group_keys=False).apply(get_clusters)
data.sample(5)

In [ ]:
def feature_dataset():
    return data